In [19]:
strategy_name ='玉皇2号'
off_start = ('close_mtx', 0)

In [20]:
import sys
sys.path.append("C:\Program Files\Tinysoft\Analyse.NET")
sys.path.append(r"C:\Users\xudong\Documents\github\coresearch\funcs")
import rschLib
import pymongo
import numpy as np
import matplotlib.pyplot as plt
import pickle
import time
import datetime
import copy
import pandas as pd
from operator import itemgetter
import TSLPy3 as tsl
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import talib
import matplotlib.dates as mdates

np.set_printoptions(formatter={'float_kind': "{:.6f}".format})
client = pymongo.MongoClient('localhost', 27017)
db = rschLib.db_quanLiang()
dbt = rschLib.db_tinySoftData()

In [21]:
#参数：
startDate = 20180101

In [22]:
def dtes2Label(dtes):
    return np.array([datetime.datetime.strptime(str(d), '%Y%m%d').date() for d in dtes])


In [23]:
plt.rcParams['font.sans-serif'] = [u'SimHei']
plt.rcParams['axes.unicode_minus'] = False
default_dpi = plt.rcParamsDefault['figure.dpi']
plt.rcParams['figure.dpi'] = default_dpi*1

In [24]:
with open(r"d:\pkl\dailyBarMtx.pkl", 'rb+') as f:
    z = pickle.load(f)
dtes = z['dtes']
dtes = np.array(dtes).astype(np.int)
tkrs = list(z['tkrs'])
open_mtx = z['open_mtx']
high_mtx = z['high_mtx']
low_mtx = z['low_mtx']
close_mtx = z['close_mtx']

k = close_mtx==0
close_mtx = rschLib.zero2pre(close_mtx)
open_mtx[k]=close_mtx[k]
high_mtx[k]=close_mtx[k]
low_mtx[k]=close_mtx[k]

name = z['name']
belong = z['belong']
shenwan1 = z['shenwan1']
shenwan2 = z['shenwan2']
shenwan3 = z['shenwan3']
vol_mtx = z['vol_mtx']
amount_mtx = z['amount_mtx']

In [25]:
v = pd.DataFrame(vol_mtx)
q = np.array(v.rolling(5,axis=1).mean())
q = np.array(q)
lb = np.zeros(vol_mtx.shape)+1
lb[:,1:]=vol_mtx[:, 1:]/q[:,:-1]
lb[np.isfinite(lb)==False]=0

C:\ProgramData\Anaconda3\envs\research\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in true_divide
  """
C:\ProgramData\Anaconda3\envs\research\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [26]:
idxTiaoKongGaoKai = np.hstack((np.zeros((open_mtx.shape[0],1))==1,((open_mtx[:,1:] / high_mtx[:,:-1]) - 1 > 0.01)))

C:\ProgramData\Anaconda3\envs\research\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\envs\research\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


In [27]:
name = list(name)
tkrs = list(tkrs)

In [28]:
Ns = 10 # 取每天量比的前多少名
Wl = 500 # 当天收盘价格位于Wl日内的高低点相对位置
Ws = 20 #  当天收盘价格位于Ws日内的高低点相对位置
priceLocL = np.array(np.zeros(close_mtx.shape)) #较长时间内的价格位置
priceLocS = np.array(np.zeros(close_mtx.shape)) #较短时间内的价格位置
preHighL = np.array(np.zeros(close_mtx.shape))
preLowL = np.array(np.zeros(close_mtx.shape))
preHighS = np.array(np.zeros(close_mtx.shape))
preLowS = np.array(np.zeros(close_mtx.shape))
idxZhangtingAtClose = np.array(np.zeros(close_mtx.shape), dtype='bool') # 收盘没有涨停
idxZhangtingAtOpen = np.array(np.zeros(close_mtx.shape), dtype='bool') # 开盘没有涨停
idxZhangtingAtClose[:, 1:] = (close_mtx[:, 1:] / close_mtx[:, :-1] - 1) > 0.095
idxZhangtingAtOpen[:, 1:] = (open_mtx[:, 1:] / close_mtx[:, :-1] - 1) > 0.095

for k in range(1,close_mtx.shape[1]):    
    #长期价格位置
    q = np.max((0, k-Wl))
    preHighL[:, k] = np.max(high_mtx[:, q:k], axis=1)
    preLowL[:, k] = np.min(low_mtx[:, q:k], axis=1)
    priceLocL[:, k] = (close_mtx[:,k-1]-preLowL[:,k])/(preHighL[:,k]-preLowL[:,k])
    #短期价格位置
    q = np.max((0, k-Ws))
    preHighS[:, k] = np.max(high_mtx[:, q:k], axis=1)
    preLowS[:, k] = np.min(low_mtx[:, q:k], axis=1)
    priceLocS[:, k] = (close_mtx[:,k-1]-preLowS[:,k])/(preHighS[:,k]-preLowS[:,k])
    

C:\ProgramData\Anaconda3\envs\research\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in true_divide
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\envs\research\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\envs\research\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: divide by zero encountered in true_divide
  del sys.path[0]
C:\ProgramData\Anaconda3\envs\research\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]
C:\ProgramData\Anaconda3\envs\research\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
C:\ProgramData\Anaconda3\envs\research\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide


In [29]:
q = list(db.tkrsInfo.find({},{
    'ticker':1,
    'circulateMarketValue20150101':1,
    'circulateMarketValue20160101':1,
    'circulateMarketValue20170101':1,
    'circulateMarketValue20180101':1,
    'circulateMarketValue20190101':1,
    'circulateMarketValue20200101':1,
    'circulateMarketValue':1
}))
cmv2016 = np.zeros(len(tkrs))
cmv2017 = np.zeros(len(tkrs))
cmv2018 = np.zeros(len(tkrs))
cmv2019 = np.zeros(len(tkrs))
cmv2020 = np.zeros(len(tkrs))
cmv = np.zeros(len(tkrs))
for x in q:
    if (x['ticker'] in list(tkrs))==False:
        continue
    if 'circulateMarketValue' in x:
        cmv[list(tkrs).index(x['ticker'])]=x['circulateMarketValue']
    if 'circulateMarketValue20160101' in x:
        cmv2016[list(tkrs).index(x['ticker'])]=x['circulateMarketValue20160101']
    if 'circulateMarketValue20170101' in x:
        cmv2017[list(tkrs).index(x['ticker'])]=x['circulateMarketValue20170101']
        cmv[list(tkrs).index(x['ticker'])]=x['circulateMarketValue']
    if 'circulateMarketValue20180101' in x:
        cmv2018[list(tkrs).index(x['ticker'])]=x['circulateMarketValue20180101']
        cmv[list(tkrs).index(x['ticker'])]=x['circulateMarketValue']
    if 'circulateMarketValue20190101' in x:
        cmv2019[list(tkrs).index(x['ticker'])]=x['circulateMarketValue20190101']
        cmv[list(tkrs).index(x['ticker'])]=x['circulateMarketValue']
    if 'circulateMarketValue20200101' in x:
        cmv2020[list(tkrs).index(x['ticker'])]=x['circulateMarketValue20200101']


In [30]:
#cmv_threshold = 100e4
#wants = list(np.array(tkrs)[cmv2020>cmv_threshold])
#off = 1
#print('选股池大小：', len(wants))
#q = [list(tkrs).index(x) for x in wants]
#idxT = [x in q for x in range(len(tkrs))]

In [31]:
import pickle
off = 1
f = open(r'd:\pkl\tagMACDDiffAboveZero.pkl','rb')
info = pickle.load(f)
idxDif = info['tag_mtx']==1




In [32]:
import pickle
off = 1
f = open(r'd:\pkl\tagCirculateMarketValueBiggerThan100Y.pkl','rb')
info = pickle.load(f)
cmv_tag = info['tag_mtx']
idxT = cmv_tag == 1



In [33]:
idxNST = [('ST' in x)==False for x in name]

In [34]:
startDate = 20180101
idxM1 = idxT & idxDif
idxM2 = idxT

ValueError: operands could not be broadcast together with shapes (4712,3892) (4715,3893) 

In [ ]:
def generate_trades(idxM, strategy_name):
    priceLocThrsMin = 0.1 # 价格大于近期的priceLocthrsMin位置
    priceLocThrsMax = 0.5 # 价格小于近期的priceLocthrsMax位置
    lbMax = 30 # 量比允许最大值
    db.strategyBackTestTrades.remove({'strategy_name':strategy_name})
    k0  = np.nonzero(dtes>startDate)[0][1]
    pnl1 = []
    pnl2 = []
    pnl3 = []
    pnl4 = []
    pnl5 = []
    tradesCount = []
    ll = list(tkrs)
    nn = list(name)
    dtesUsed = []
    f = open("output.txt", "w")
    sp = close_mtx.shape[1]-1
    for k in range(k0, close_mtx.shape[1]):
        # 跳空高开， 价格位置
        #idxQ = (idxTiaoKongGaoKai[:,k]==True) & idxT & (priceLocL[:, k]>0.1) & (priceLocL[:, k]<0.9)
        # 跳空高开
        idxQ = (idxTiaoKongGaoKai[:,k]==True) & idxM[:, k] & (priceLocL[:, k]<0.5)
        #idxQ = (idxTiaoKongGaoKai[:,k]==True) & idxT & (priceLocL[:, k]<0.2)
        #选择高量比
        m = np.nonzero(idxQ==True)[0]
        lbm = lb[m,k]
        lidx = m[np.argsort(lbm)[::-1]] #得到哪些股票在idxQ中存在且量比排名前十
        if (len(lidx)>10):
            lidx=lidx[:10]
        idxQ2 = np.zeros(idxQ.shape)==1
        idxQ2[lidx]=True

        t  = np.nonzero(idxQ2)[0]
        m = np.array(tkrs)[t]
        #qt = list(dbt.minuteBarStock.find({'ticker':{'$in': list(m)},'dateAsInt':int(dtes[k]),"offSetFromMidNight" :  5370e4, 'lb':{'$lte':50}}, {'ticker':1, 'close':1, 'open':1, 'sectional_open':1, 'lb':1, 'sale1':1, 'dateTime':1, 'StockName':1}).sort('lb',-1).limit(10))
        qt = list(dbt.minuteBarStock.find({'ticker':{'$in': list(m)},'dateAsInt':int(dtes[k]),"offSetFromMidNight" :  5370e4, 'lb':{'$lte':50}}, {'ticker':1, 'close':1, 'open':1, 'sectional_open':1, 'lb':1, 'sale1':1, 'dateTime':1, 'StockName':1}).sort('lb',-1))
        q = []
        qs = []
        for x in qt:
            if (x['sale1']==0):
                print('忽略涨停股：',x['StockName'],dtes[k], x['dateTime'])
                continue
            if (x['close']/x['sectional_open']-1 < 0):
                continue
            if (k<sp) & (idxZhangtingAtOpen[ll.index(x['ticker']), np.min((k+1, sp))]==True):
                print(x['ticker'], dtes[k],' 第二日开盘涨停不买')
                continue
            if x['ticker'] in qs:
                continue
            q.append(x)
            qs.append(x['ticker'])
        if (len(q)==0):
            continue
        mk = [x['ticker'] for x in q]
        q0 = list(dbt.minuteBarStock.find({'ticker':{'$in': mk},'dateAsInt':int(dtes[k]),"offSetFromMidNight" : 34260000}, {'ticker':1, 'close':1, 'lb':1}))
        qc = list(dbt.minuteBarStock.find({'ticker':{'$in': mk},'dateAsInt':int(dtes[k]),"offSetFromMidNight" : 53940000}, {'ticker':1, 'close':1, 'lb':1}))
        m0 = [x['ticker'] for x in q0]
        mc = [x['ticker'] for x in qc]
        index0 = [m0.index(x) for x in mk]
        indexc = [mc.index(x) for x in mk]
        close0930 = np.array([x['close'] for x in q0])[index0]
        close1500 = np.array([x['close'] for x in qc])[indexc]
        close0935  = np.array([x['close'] for x in q])
        sn = [x['ticker'] for x in q]
        idx = [ll.index(x['ticker']) for x in q]
        lbm = np.array([x['lb'] for x in q])
        pLocL = priceLocL[idx, k]
        pLocS = priceLocS[idx, k]
        close_today = close_mtx[idx, k]  # 今天收盘
        open_today = open_mtx[idx, k]    # 今天开盘
        close_tomorrow = close_mtx[idx, np.min((k+1,sp))] # 明天收盘
        if k+1<sp: # 明天开盘
            open_tomorrow = open_mtx[idx, k+1]
        else:
            open_tomorrow = close_mtx[idx, sp]
        if k+2<sp:
            open_p3 = open_mtx[idx, k+2]   # 后天开盘
        else:
            open_p3 = close_mtx[idx, sp]   # 后天开盘
        close_p3 = close_mtx[idx, np.min((k+2,sp))]  # 后天收盘
        close_yesterday = close_mtx[idx, k-1]
        close_yesterday2 = close_mtx[idx, k-2]
        isZhangTing = idxZhangtingAtClose[idx, k]

        # r1: 第一天入场到收盘
        # r2: 第一天收盘到第二天开盘
        # r3: 第二天开盘到第二天收盘
        # r4: 第二天收盘到第三天开盘
        # r5: 第三天开盘到第三天收盘
        r1 = np.mean(close1500/close0935 - 1)
        r2 = np.mean(open_tomorrow/close_today  - 1)
        r3 = np.mean(close_tomorrow/open_tomorrow - 1)
        r4 = np.mean(open_p3/close_tomorrow - 1)
        r5 = np.mean(close_p3/open_p3 - 1)
        pnl1.append(r1)
        pnl2.append(r2)
        pnl3.append(r3)
        pnl4.append(r4)
        pnl5.append(r5)
        tradesCount.append(len(close1500))
        dtesUsed.append(dtes[k])
        for (iq,t) in enumerate(sn):
            s = nn[ll.index(t)] + ','+str(dtes[k])+',量比,'+str(np.round(lbm[iq]))+\
            ',价格位置（长期),'+str(np.round(1e4*pLocL[iq])/1e2)+\
            ',价格位置（短期),'+str(np.round(1e4*pLocS[iq])/1e2)+\
            ',cmv2019,'+str(cmv2019[ll.index(t)])+\
            ',cmv2020,'+str(cmv2020[ll.index(t)])+\
            ',cmv2018,'+str(cmv2018[ll.index(t)])+\
            ',第一天涨幅,'+str(close_today[iq]/close_yesterday[iq]-1)+\
            ',开盘涨幅（9:30-9:35）,'+str(close0935[iq]/close0930[iq]-1)+\
            ',入场后到收盘,'+str(close_today[iq]/close0935[iq]-1)+\
            ',第二天开盘涨幅,'+str(open_tomorrow[iq]/close_today[iq]-1)+\
            ',第二天日内涨幅,'+str(close_tomorrow[iq]/open_tomorrow[iq]-1)+\
            ',第二天收盘至第三天开盘,'+str(open_p3[iq]/close_tomorrow[iq]-1)+\
            ',第三天开盘至收盘,'+str(close_p3[iq]/open_p3[iq]-1)+\
            ',昨日涨幅,'+str(close_yesterday[iq]/close_yesterday2[iq]-1)+\
            ',跳空高开幅度,'+str(open_today[iq]/close_yesterday[iq]-1)+\
            ',买入价,'+str(open_tomorrow[iq])+\
            ',卖出价,'+str(close_p3[iq])+\
            ',利润,'+str(close_p3[iq]/open_tomorrow[iq]-1)+\
            ',是否涨停,'+str(isZhangTing[iq])
            f.write(s+'\n')
            print(nn[ll.index(t)] + ','+str(dtes[k])+',量比,'+str(np.round(lbm[iq]))+',进场价格:'+str(open_tomorrow[iq])+',出场价格:'+str(close_p3[iq])+',利润,'+str(close_p3[iq]/open_tomorrow[iq]-1))
            db.strategyBackTestTrades.insert_one({
                'ticker':t,
                'name':nn[ll.index(t)], 
                'dateIn':int(dtes[k]), 
                'strategy_name':strategy_name})
    f.close()
    pnl1 = np.array(pnl1)
    pnl2 = np.array(pnl2)
    pnl3 = np.array(pnl3)
    pnl4 = np.array(pnl4)
    pnl5 = np.array(pnl5)
    pnl1[np.isfinite(pnl1)==False]=0
    pnl2[np.isfinite(pnl2)==False]=0
    pnl3[np.isfinite(pnl3)==False]=0
    pnl4[np.isfinite(pnl4)==False]=0
    pnl5[np.isfinite(pnl5)==False]=0
    plt.plot(tradesCount)
    plt.plot(dtes2Label(dtesUsed), np.cumsum(pnl1), 'k')
    plt.plot(dtes2Label(dtesUsed), np.cumsum(pnl2), 'b')
    plt.plot(dtes2Label(dtesUsed), np.cumsum(pnl3), 'r')
    plt.plot(dtes2Label(dtesUsed), np.cumsum(pnl4), 'c')
    plt.plot(dtes2Label(dtesUsed), np.cumsum(pnl5), 'm')
    plt.legend(['第一天入场到第一天收盘', '第一天收盘到第二天开盘','第二天开盘到第二天收盘','第二天收盘到第三天开盘','第三天开盘到第三天收盘'])
    plt.grid()
    rschLib.drawPNL(dtesUsed, pnl3+pnl4+pnl5,  dtes, strategy_name, toDatabase='yes')
    rschLib.saveOffStart(strategy_name, off_start)
    return

In [ ]:
generate_trades(idxM1, strategy_name)
generate_trades(idxM2, strategy_name+'原始')
